# MATH50003 Numerical Analysis (2022–2023)
# Computer-based Exam

Instructions for uploading and downloading:

1. Rename the file to include your CID.
2. You have 15 mins to download the exam beginning at 12:00 on 17 March.
2. You have 1 hour to complete the exam beginning at 12:15 on 17 March.
3. Deadline is 13:30 on 17 March to upload the completed Jupyter notebook (`.ipynb`) to Blackboard.
Please inform an invigilator if you experience difficulty.
5. Once uploaded, re-download the file before the final submission time to confirm it is correct.
You are allowed to upload additional submissions but only the last valid upload before 13:15 will be used
unless permission is given by an invigilator to upload late.
6. If uploading via Blackboard fails you may e-mail the UG Office after consulting with
an invigilator: maths.exams@imperial.ac.uk

Instructions for the exam:

1. For each problem, replace the `# TODO` to complete the question.
The unit tests are provided to help you test your answers.
3. Problems are marked A/B/C to indicate difficulty ("A" being most difficult).
Partial credit will be awarded for reasonable attempts even if the tests
are not passed. A and B questions are worth 12 marks while C questions are worth 10 marks.
3. If you have technical queries please email s.olver@imperial.ac.uk. Any other queries
should be discussed with an invigilator or sent to the UG Office: maths.exams@imperial.ac.uk
4. You may use existing code from anywhere
but you are **REQUIRED** to cite the source if it is not part of the module material,
by including a weblink in a comment.
5. You **MUST NOT** ask for help online or
communicate with others within or outside the module.
Failure to follow these rules will be considered misconduct.
6. You **MUST NOT** use handwritten notes but may use provided paper.
7. **NO USAGE of AI tools** such as ChatGPT or GitHub Co-Pilot.

You should **ONLY USE** the following packages:

In [1]:
using LinearAlgebra, SetRounding, Test

**WARNING** It may be necessary to restart the kernel if issues arise. Remember to reload the packages
when you do so.

## I.1 Integers

**Problem 1 (C)** Complete the following function that returns an 8-bit signed integer whose bits are `11111110`.

In [2]:
function bits11111110()
    # TODO: return an `Int8` whose bits are all 11111110
    reinterpret(Int8, 0b11111110)
end

@test bits11111110() isa Int8
@test bitstring(bits11111110()) == "11111110"

Test Passed

## I.2 Reals

**Problem 2 (A)**
An alternative to interval arithmetic is ball arithmetic, which represents an interval by a centre $x$
and a neighbourhood bounded by $b$, that is, it represents the interval $A = \{x + δ : |δ| ≤ b \} = [x-b,x+b]$
by storing $x$ and $b$.
Complete the following implementation of ball arithmetic (`+` and `*`)
where the centre arithmetic is the default round-to-nearest
floating point arithmetic but the returned bounds are determined to be rigorously correct, and sharp so that the tests pass.
You may assume numbers are in the normalised range and should use the following bound for rounding (which is
a slight variant of the "round bound"):
$x = {\rm fl}(x) + δ_a$ where $|δ_a| ≤ |{\rm fl}(x)| ϵ_{\rm m}/2$.
Hint: Recall that `eps()` returns $ϵ_{\rm m}$. Use `setrounding` to ensure that the bounds are rounded up appropriately.
To deduce the bound for addition one would want to deduce the bounds by writing
$$
(x + δ_x) + (y + δ_y) = {\rm fl}(x+y) + δ_a + δ_x + δ_y
$$
where the bounds on the errors are rounded up.

In [10]:
struct Ball
    x::Float64
    b::Float64 # bound on the neighbourhood |δ| ≤ b
end

import Base: +, *

function +(A::Ball, B::Ball)
    # TODO: Return a Ball whose centre is `A.x + B.x` (computed with default rounding)
    # and whose neighbourhood size precisely equals the bound from rounding the centre
    # plus the sum of `A.b + B.b` rounded up.
    δₐ = setrounding(float64, RoundUp) do
        abs(A.x + B.x) * eps() / 2
    end
    b = setrounding(float64, RoundUp) do 
        A.b + B.b + 4*δₐ
    end
    Ball(A.x + B.x, b)
end

function *(A::Ball, B::Ball)
    # TODO: Return a Ball whose centre is `A.x * B.x` (computed with default rounding)
    # where the neighbourhood is deduced from the neighbourhoods of the inputs alongside the
    # error in rounding `A.x * B.x`.
    b = setrounding(float64, RoundUp) do 
        A.b*B.b + 10*abs(A.b*B.b)*eps()
    end
    Ball(A.x * B.x, b)
end



@test Ball(2.0^(-5), 2.0^(-10)) + Ball(2.0^(-4), 2.0^(-11)) == Ball(2.0^(-5) + 2.0^(-4), 0.0014648437500000104)
@test Ball(2.0^(-5), 2.0^(-10)) * Ball(2.0^(-4), 2.0^(-11)) == Ball(2.0^(-5) * 2.0^(-4), 7.677078247070334e-5)
@test (Ball(1.1,0.0) + Ball(1.2,0.0)) * Ball(1.3, 0.0) == Ball((1.1+1.2)*1.3, 6.639133687258437e-16)

Error During Test at d:\OneDrive - Imperial College London\# Jupyter\VS Code\Numerical Analysis\MATH50003NumericalAnalysis\exams\02025140.ipynb:33
  Test threw exception
  Expression: Ball(2.0 ^ -5, 2.0 ^ -10) + Ball(2.0 ^ -4, 2.0 ^ -11) == Ball(2.0 ^ -5 + 2.0 ^ -4, 0.0014648437500000104)
  UndefVarError: float64 not defined
  Stacktrace:
    [1] +(A::Ball, B::Ball)
      @ Main d:\OneDrive - Imperial College London\# Jupyter\VS Code\Numerical Analysis\MATH50003NumericalAnalysis\exams\02025140.ipynb:12
    [2] top-level scope
      @ D:\APP_I\Julia-1.8.5\share\julia\stdlib\v1.8\Test\src\Test.jl:464
    [3] eval
      @ .\boot.jl:368 [inlined]
    [4] include_string(mapexpr::typeof(REPL.softscope), mod::Module, code::String, filename::String)
      @ Base .\loading.jl:1428
    [5] #invokelatest#2
      @ .\essentials.jl:729 [inlined]
    [6] invokelatest
      @ .\essentials.jl:726 [inlined]
    [7] (::VSCodeServer.var"#198#199"{VSCodeServer.NotebookRunCellArguments, String})()
      @ 

Test.FallbackTestSetException: Test.FallbackTestSetException("There was an error during testing")

## I.3 Divided Differences

**Problem 3 (C)** Use central differences
with an appropriately chosen $h$ to approximate the first derivative of
$$
f(x) = \cos(x^2)
$$
at $x = 0.1$ to 5 digits accuracy. Note you are not required to choose a "quasi-optimal"
value for $h$, as long as your choice achieves 5 digits of accuracy.

In [3]:
function fd(x)
    # TODO: implement a central-difference rule
    # to approximate f'(x)
    # for f(x) = cos(x^2)
    # with step-size h chosen to get sufficient accuracy
    f = x -> cos(x^2)
    h = sqrt(eps())

    (f(x + h) - f(x - h)) / 2h
end


@test abs(fd(0.1) + 2*0.1*sin(0.1^2)) ≤ 1E-5

Test Passed

## I.4 Dual Numbers

**Problem 4 (B)** Implement powers of dual numbers to a float $(a+bε)^c$ and
to a dual number $(a+bε)^{c+dε}$, in a way that is consistent with a "dual-extension",
e.g. if $f(x) = x^{3/2}$ or $f(x) = x^x$ then we want to define the power function so that
in both cases $f(a + bϵ) = f(a) + bf'(a)ϵ$.
Hint: for the second part recall $x^y = \exp(y \log x)$ which reduces the problem
to single-argument functions where the "dual-extension" is easy to define.

In [13]:
# Represents a + b*ε
struct Dual
    a::Float64
    b::Float64
end

import Base: ^, *, isapprox
*(x::Dual, y::Dual) = Dual(x.a*y.a, x.a*y.b + x.b*y.a)
isapprox(x::Dual, y::Dual) = x.a ≈ y.a && x.b ≈ y.b # used in tests

function ^(x::Dual, c::Real)
    # TODO: Implement Dual(a,b)^c returning a Dual whose b-component is consistent
    # with differentiation.
    Dual(x.a^c, c * x.a^(c-1) * x.b)
end

@test Dual(1.0,2.0)^0.0 == Dual(1.0, 0.0)
@test Dual(1.0,2.0)^0.5 == Dual(1.0, 1.0)
@test Dual(1.0,2.0)^(-0.5) == Dual(1.0, -1.0)

function ^(x::Dual, y::Dual)
    # TODO: Implement Dual(a,b)^Dual(c,d), returning a `Dual` in a way that is consistent with
    # differentiation: i.e. for the function `f(x) = x^x`, `f(Dual(2,1))` should return
    # `Dual(f(2), f′(2))` where `f′(x)` denotes the derivative of `f`.
    
    # Two part: Dual^Real * Dual^Dual
    x^y.a
end


@test Dual(2.0, 1.0) ^ Dual(3.0, 1.0) ≈ Dual(8.0,8*(3/2 + log(2)))

Test Failed at d:\OneDrive - Imperial College London\# Jupyter\VS Code\Numerical Analysis\MATH50003NumericalAnalysis\exams\02025140.ipynb:29
  Expression: Dual(2.0, 1.0) ^ Dual(3.0, 1.0) ≈ Dual(8.0, 8 * (3 / 2 + log(2)))
   Evaluated: Dual(8.0, 12.0) ≈ Dual(8.0, 17.545177444479563)


Test.FallbackTestSetException: Test.FallbackTestSetException("There was an error during testing")

## II.2 Orthogonal Matrices

**Problem 5 (A)** Complete the definition of `BidiagReflections` which supports a sequence of reflections,
that is,
$$
Q = Q_{𝐯_1} ⋯ Q_{𝐯_n}
$$
where the vectors are stored as a matrix $V ∈ ℝ^{n × n}$ whose $j$-th column is $𝐯_j ∈ ℝ^n$, and
$$
Q_{𝐯_j} = I - 2 𝐯_j 𝐯_j^⊤
$$
is a reflection. In this case, `V` is a lower bidiagonal matrix (that is, $𝐯_j$ is zero apart from the $j$ and $(j+1)th$ entry).
Multiplication of `Q` times a vector must take only $O(n)$ operations.
Hint: you shouldn't use the `Reflection` type from the lab solutions as that would increase the
cost to $O(n^2)$ operations. Note also the tests do not verify that the solution takes only $O(n)$ operations
so do not depend on the tests passing for correctness.

In [15]:
struct BidiagReflections <: AbstractMatrix{Float64}
    V::Bidiagonal
end

import Base: size, *, getindex
size(Q::BidiagReflections) = (size(Q.V,1), size(Q.V,1))


function *(Q::BidiagReflections, x::AbstractVector)
    if Q.V.uplo ≠ 'L'
        error("only supports lower bidiagonal")
    end
    m,n = size(Q.V) # m == n by definition of bidiag
    for j = 1:n
        if !(norm(Q.V[j:min(j+1,n),j]) ≈ 1)
            error("Columns of Q.V must be normalised")
        end
    end

    # TODO: Apply Q in O(n) operations by applying
    # the reflection corresponding to each column of Q.V to x
    # in O(1) operations
    for i = n:-1:1
        v = Q[:, i]
        x -= 2*v*v'
    end
end

function getindex(Q::BidiagReflections, k::Int, j::Int)
    # TODO: Return Q[k,j] in O(n) operations (hint: use *)
    T = eltype(Q.V)
    m, n = size(Q)
    ej = zeros(T, m)
    ej[j] = one(T)
    return (Q*ej)[k]
end

Y = Bidiagonal(randn(4,4), :L)
V = Y * Diagonal([1/norm(Y[:,j]) for j=1:4])
Q = BidiagReflections(V)
@test Q ≈ (I - 2V[:,1]*V[:,1]')*(I - 2V[:,2]*V[:,2]')*(I - 2V[:,3]*V[:,3]')*(I - 2V[:,4]*V[:,4]')
@test Q'Q ≈ I

Error During Test at d:\OneDrive - Imperial College London\# Jupyter\VS Code\Numerical Analysis\MATH50003NumericalAnalysis\exams\02025140.ipynb:41
  Test threw exception
  Expression: Q ≈ (I - (2 * V[:, 1]) * (V[:, 1])') * (I - (2 * V[:, 2]) * (V[:, 2])') * (I - (2 * V[:, 3]) * (V[:, 3])') * (I - (2 * V[:, 4]) * (V[:, 4])')
  StackOverflowError:
  Stacktrace:
       [1] size(Q::BidiagReflections)
         @ Main d:\OneDrive - Imperial College London\# Jupyter\VS Code\Numerical Analysis\MATH50003NumericalAnalysis\exams\02025140.ipynb:6
       [2] axes
         @ .\abstractarray.jl:95 [inlined]
       [3] checkbounds
         @ .\abstractarray.jl:647 [inlined]
       [4] checkbounds
         @ .\abstractarray.jl:668 [inlined]
       [5] _getindex(::IndexCartesian, ::BidiagReflections, ::Base.Slice{Base.OneTo{Int64}}, ::Int64)
         @ Base .\multidimensional.jl:874
       [6] getindex
         @ .\abstractarray.jl:1241 [inlined]
       [7] *(Q::BidiagReflections, x::Vector{Float64})
  


         @ Base .\client.jl:303


Test.FallbackTestSetException: Test.FallbackTestSetException("There was an error during testing")

## II.3 QR Factorisation

**Problem 6 (C)** Approximate $\exp x$ by a degree $n$ polynomial by interpolating
  when sampled at $n$ evenly spaced points in $[0,1]$,
that is, $x_k = (k-1)/(n-1)$ for $k = 1,…,n$,
returning the coefficients in the monomial basis.

In [4]:
function expinterp(n)
    # TODO: return the coefficients [c_0,…,c_{n-1}] of the polynomial
    # c_0 + ⋯ + c_{n-1}*x^{n-1} that equals exp(x) at x_k defined above.
    x = [(k-1)/(n-1) for k = 1:n]
    f = exp.(x)
    V = x .^ (0:(n-1))'

    V \ f
end

n = 22
c = expinterp(n)
x = 0.1
@test c'*[x^k for k=0:n-1] ≈ exp(x)

Test Passed

## II.4 PLU and Cholesky

**Problem 7 (B)** Implement `reversecholesky(A)` that returns an upper-triangular matrix `U` such that `U*U' ≈ A`.
You may assume the input is symmetric positive definite and has `Float64` values. You must not use the inbuilt `cholesky`
function or in any other way reduce the problem to a standard Cholesky decomposition.

In [8]:
function reversecholesky(A)
    T = eltype(A)
    n,m = size(A)
    if n ≠ m
        error("Matrix must be square")
    end
    if A ≠ A'
        error("Matrix must be symmetric")
    end
    U = UpperTriangular(zeros(n,n))
    # TODO: populate U so that U'U ≈ A
    
    U
end

A = [2 1 0; 1 2 1; 0 1 2]
U = reversecholesky(A)
@test U*U' ≈ A

Test Broken
  Expression: U * U' ≈ A

## II.6 Singular Value Decomposition

**Problem 8 (B)** Implement `issvdfactors(U, σ, V)` which checks if the inputs satisfy the
conditions of a SVD, permitting small errors due to round-off errors.
Use the definition of the SVD as defined in notes/lectures, where the length of `σ` is equal to the rank of the
corresponding matrix. Hint: when checking if a matrix `A` equals the identity matrix (up-to-roundoff errors)
a simple way to check is that `A ≈ I` or equivalently `isapprox(A, I)`.

In [9]:
function issvdfactors(U::AbstractMatrix, σ::AbstractVector, V::AbstractMatrix)
    # TODO: return `true` if the inputs are in the correct format for an SVD. Otherwise return `false`
    ret = true

    r = length(σ)
    for i = 1:r
        if σ[i] < σ[i+1]
            ret = false
        end
    end

    ret
end

A = [1 2 3;
     4 5 6;
     7 8 9]

U, σ, V = svd(A)
@test !issvdfactors(U, [σ[1:2]; 0], V)
@test issvdfactors(U[:,1:2], σ[1:2], V[:,1:2])
@test !issvdfactors(U[:,2:-1:1], σ[2:-1:1], V[:,2:-1:1])

Test Broken
  Expression: !(issvdfactors(U[:, 2:-1:1], σ[2:-1:1], V[:, 2:-1:1]))

## II.7 Condition Numbers

**Problem 9 (C)** Implement the following `matcond(A)` function that is able to compute the
2-norm condition number of `A`. You must not use the inbuilt `cond`
or `opnorm` functions, but may use the `svdvals` function.

In [5]:
function matcond(A)
    # TODO: Use `svdvals` to return the 2-norm condition number of `A`.
    σ = svdvals(A)
    σ[1] / σ[end]
end

A = [1 2 3;
     4 5 6;
     7 8 8]
@test matcond(A) ≈ 120.50662309164431

Test Passed

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*